# Data Preprocessing

In [4]:
import pandas
loadeddata = pandas.read_csv('HDBPropertyInformation.csv')
loadeddata.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0


In [6]:
loadeddata.isna().any()

blk_no                   False
street                   False
max_floor_lvl            False
year_completed           False
residential              False
commercial               False
market_hawker            False
miscellaneous            False
multistorey_carpark      False
precinct_pavilion        False
bldg_contract_town       False
total_dwelling_units     False
1room_sold               False
2room_sold               False
3room_sold               False
4room_sold               False
5room_sold               False
exec_sold                False
multigen_sold            False
studio_apartment_sold    False
1room_rental             False
2room_rental             False
3room_rental             False
other_room_rental        False
dtype: bool

# Feature Engineering

# Model Selection

## Linear Regression

## Random Forest

# Model Evaluation

# Hyperparameter Tuning

# Model Interpretation